In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fare-classification-msc2020/meta_data.csv
/kaggle/input/inputdata/test.csv
/kaggle/input/inputdata/train.csv


In [3]:
train = pd.read_csv('../input/inputdata/train.csv')
test = pd.read_csv('../input/inputdata/test.csv')

In [ ]:
train.head()

In [4]:
predictor_cols = ['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup', 'fare']
train_X = train[predictor_cols]
test_X = test[predictor_cols]

#train_y.fillna(train_y.mean(), inplace=True)
train.fillna(train_X.mean(), inplace=True)
test.fillna(test_X.mean(), inplace=True)
train['num_label'] = False
train['direct_distance'] = 0.0
test['direct_distance'] = 0.0

train['pickup_time'] = pd.to_datetime(train['pickup_time'], errors='coerce')
test['pickup_time'] = pd.to_datetime(test['pickup_time'], errors='coerce')

train['drop_time'] = pd.to_datetime(train['drop_time'], errors='coerce')
test['drop_time'] = pd.to_datetime(test['drop_time'], errors='coerce')

train['pick_hour'] = train['pickup_time'].dt.hour
train['drop_hour'] = train['drop_time'].dt.hour

test['pick_hour'] = test['pickup_time'].dt.hour
test['drop_hour'] = test['drop_time'].dt.hour

for index, row in train.iterrows():
    lat1, lng1, lat2, lng2 = map(np.radians, (row['drop_lat'], row['drop_lon'], row['pick_lat'], row['pick_lon']))
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * 6371 * np.arcsin(np.sqrt(d))
    train.at[index,'direct_distance'] = 2 * 6371 * np.arcsin(np.sqrt(d))
    #d = (((row['drop_lon'] - row['pick_lon'])**2)+((row['drop_lat'] - row['pick_lat'])**2))
    #train.at[index,'direct_distance'] = 2 * 6371 * np.arcsin()
    
for index, row in test.iterrows():
    lat1, lng1, lat2, lng2 = map(np.radians, (row['drop_lat'], row['drop_lon'], row['pick_lat'], row['pick_lon']))
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * 6371 * np.arcsin(np.sqrt(d))
    test.at[index,'direct_distance'] = 2 * 6371 * np.arcsin(np.sqrt(d))
    
train['fare_for_duration'] = (train['fare'] - train['meter_waiting_fare'])/train['duration']
#train['fare_for_distance'] = (train['fare'] - train['meter_waiting_fare'])/train['direct_distance']
train['fare_for_time'] = (train['fare'] - train['meter_waiting_fare'])/(train['duration']+train['meter_waiting_till_pickup'])
#train['distance_for_time'] = train['direct_distance']/train['duration']
train['meter_waiting_to_duration'] = train['meter_waiting']/train['duration']
train['additional_fare_to_duration'] = train['additional_fare']/train['duration']
#train['additional_fare_to_distance'] = train['additional_fare']/train['direct_distance']
train['additional_fare_to_fare'] = train['additional_fare']/(train['fare']+train['additional_fare'])
train['mtr_wating_fare_to_waiting_duration'] = train['meter_waiting_fare']/(train['meter_waiting']+train['meter_waiting_till_pickup'])
train['additional_fare_to_full_time'] = train['additional_fare']/(train['meter_waiting']+train['meter_waiting_till_pickup']+train['duration'])
train['full_waiting_fare_to_full_time'] = (train['meter_waiting_fare']+train['additional_fare'])/(train['meter_waiting']+train['meter_waiting_till_pickup']+train['duration'])
train['net_fare_for_durtion'] = (train['fare'] - (train['additional_fare']+train['meter_waiting_fare']))/train['duration']


test['fare_for_duration'] = (test['fare'] - test['meter_waiting_fare'])/test['duration']
#test['fare_for_distance'] = (test['fare'] - test['meter_waiting_fare'])/test['direct_distance']
test['fare_for_time'] = (test['fare'] - test['meter_waiting_fare'])/(test['duration']+test['meter_waiting_till_pickup'])
#test['distance_for_time'] = test['direct_distance']/test['duration']
test['meter_waiting_to_duration'] = test['meter_waiting']/test['duration']
test['additional_fare_to_duration'] = test['additional_fare']/test['duration']
#test['additional_fare_to_distance'] = test['additional_fare']/test['direct_distance']
test['additional_fare_to_fare'] = test['additional_fare']/(test['fare']+test['additional_fare'])
test['mtr_wating_fare_to_waiting_duration'] = test['meter_waiting_fare']/(test['meter_waiting']+test['meter_waiting_till_pickup'])
test['additional_fare_to_full_time'] = test['additional_fare']/(test['meter_waiting']+test['meter_waiting_till_pickup']+test['duration'])
test['full_waiting_fare_to_full_time'] = (test['meter_waiting_fare']+test['additional_fare'])/(test['meter_waiting']+test['meter_waiting_till_pickup']+test['duration'])
test['net_fare_for_durtion'] = (test['fare'] - (test['additional_fare']+test['meter_waiting_fare']))/test['duration']


train.replace([np.inf, -np.inf], np.nan)
test.replace([np.inf, -np.inf], np.nan)
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

train['fare_for_duration'] = train['fare_for_duration'].astype(np.float32)
#train['fare_for_distance'] = train['fare_for_distance'].astype(np.float32)
train['fare_for_time'] = train['fare_for_time'].astype(np.float32)
#train['distance_for_time'] = train['distance_for_time'].astype(np.float32)
train['meter_waiting_to_duration'] = train['meter_waiting_to_duration'].astype(np.float32)
train['additional_fare_to_duration'] = train['additional_fare_to_duration'].astype(np.float32)
#train['additional_fare_to_distance'] = train['additional_fare_to_distance'].astype(np.float32)
train['additional_fare_to_fare'] = train['additional_fare_to_fare'].astype(np.float32)
train['mtr_wating_fare_to_waiting_duration'] = train['mtr_wating_fare_to_waiting_duration'].astype(np.float32)
train['additional_fare_to_full_time'] = train['additional_fare_to_full_time'].astype(np.float32)
train['full_waiting_fare_to_full_time'] = train['full_waiting_fare_to_full_time'].astype(np.float32)
train['net_fare_for_durtion'] = train['net_fare_for_durtion'].astype(np.float32)

test['fare_for_duration'] = test['fare_for_duration'].astype(np.float32)
#test['fare_for_distance'] = test['fare_for_distance'].astype(np.float32)
test['fare_for_time'] = test['fare_for_time'].astype(np.float32)
#test['distance_for_time'] = test['distance_for_time'].astype(np.float32)
test['meter_waiting_to_duration'] = test['meter_waiting_to_duration'].astype(np.float32)
test['additional_fare_to_duration'] = test['additional_fare_to_duration'].astype(np.float32)
#test['additional_fare_to_distance'] = test['additional_fare_to_distance'].astype(np.float32)
test['additional_fare_to_fare'] = test['additional_fare_to_fare'].astype(np.float32)
test['mtr_wating_fare_to_waiting_duration'] = test['mtr_wating_fare_to_waiting_duration'].astype(np.float32)
test['additional_fare_to_full_time'] = test['additional_fare_to_full_time'].astype(np.float32)
test['full_waiting_fare_to_full_time'] = test['full_waiting_fare_to_full_time'].astype(np.float32)
test['net_fare_for_durtion'] = test['net_fare_for_durtion'].astype(np.float32)

for index, row in train.iterrows():
    if(row['label'] == "correct"):
        train.at[index, 'num_label'] = 1
    else:
        train.at[index, 'num_label'] = 0

In [5]:
# 'fare_for_distance',
# 'distance_for_time', 'additional_fare_to_distance', 'distance_for_time', 'only_fare_for_duration',
predictor_cols = ['additional_fare', 'net_fare_for_durtion', 'full_waiting_fare_to_full_time', 'additional_fare_to_full_time', 'mtr_wating_fare_to_waiting_duration', 'fare_for_time', 'meter_waiting_to_duration', 'fare_for_duration', 'additional_fare_to_duration', 'additional_fare_to_fare', 'pick_hour', 'drop_hour', 'pick_lat', 'pick_lon', 'direct_distance', 'duration', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup', 'fare']
train_X_ = train[predictor_cols]
test_X = test[predictor_cols]
train_X_.replace([np.inf, -np.inf], np.nan)
test_X.replace([np.inf, -np.inf], np.nan)
train_X_.fillna(train_X_.mean(), inplace=True)
test_X.fillna(test_X.mean(), inplace=True)

train_y_ = train.num_label

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [6]:
train_X_

,additional_fare,net_fare_for_durtion,full_waiting_fare_to_full_time,additional_fare_to_full_time,mtr_wating_fare_to_waiting_duration,fare_for_time,meter_waiting_to_duration,fare_for_duration,additional_fare_to_duration,additional_fare_to_fare,pick_hour,drop_hour,pick_lat,pick_lon,direct_distance,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare
0,10.500000,0.311535,0.011006,0.011006,0.000000,0.301024,0.067146,0.324125,0.012590,0.037391,0,0,6.86252,79.8993,5.092770,834.000000,56.000000,0.000000,64.000000,270.32
1,10.500000,0.236852,0.010802,0.010802,0.000000,0.213892,0.059418,0.250126,0.013274,0.050396,0,1,6.88589,79.8984,3.168058,791.000000,47.000000,0.000000,134.000000,197.85
2,10.500000,0.267838,0.008550,0.008550,0.000000,0.262753,0.073597,0.277498,0.009660,0.033639,1,1,6.90839,79.8651,6.305395,1087.000000,80.000000,0.000000,61.000000,301.64
3,10.500000,0.093873,0.027923,0.011206,0.046206,0.100054,0.453177,0.111432,0.017559,0.113147,2,2,6.92570,79.8895,0.861946,598.000000,271.000000,15.663800,68.000000,82.30
4,13.719651,0.183581,0.018727,0.005613,0.043231,0.179765,0.369423,0.191638,0.008057,0.036870,3,3,6.87441,79.8615,8.147782,1702.858077,629.074231,32.057666,112.466832,358.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17171,10.500000,0.217587,0.011521,0.007598,0.009967,0.149603,0.110979,0.230117,0.012530,0.050297,22,22,7.29073,80.6367,2.105376,838.000000,93.000000,5.421900,451.000000,198.26
17172,10.500000,0.265332,0.004011,0.004011,0.000000,0.265402,0.198977,0.270214,0.004881,0.017745,23,23,6.90569,79.8516,10.868377,2151.000000,428.000000,0.000000,39.000000,581.23
17173,10.500000,0.249810,0.027487,0.027487,0.000000,0.204290,0.034221,0.289734,0.039924,0.121107,23,23,7.09210,79.9000,1.045518,263.000000,9.000000,0.000000,110.000000,76.20
17174,10.500000,0.143135,0.008140,0.008140,0.000000,0.113455,0.134033,0.155373,0.012238,0.073013,23,23,6.94540,79.8768,2.879077,858.000000,115.000000,0.000000,317.000000,133.31


In [7]:
test_X

,additional_fare,net_fare_for_durtion,full_waiting_fare_to_full_time,additional_fare_to_full_time,mtr_wating_fare_to_waiting_duration,fare_for_time,meter_waiting_to_duration,fare_for_duration,additional_fare_to_duration,additional_fare_to_fare,pick_hour,drop_hour,pick_lat,pick_lon,direct_distance,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,fare
0,10.5,0.299049,0.011624,0.009425,0.012887,0.267557,0.045455,0.310413,0.011364,0.035027,0,0,6.83454,79.8750,6.705702,924,42,2.44860,148,289.27
1,10.5,0.447682,0.002408,0.002408,0.000000,0.440714,0.004707,0.450153,0.002471,0.005460,1,2,6.91168,79.8723,41.558513,4249,20,0.00000,91,1912.70
2,10.5,0.245387,0.007191,0.005738,0.009564,0.248471,0.164304,0.252153,0.006765,0.025958,5,5,6.92145,79.8478,5.916678,1552,255,2.65880,23,394.00
3,10.5,0.311299,0.015533,0.015533,0.000000,0.233818,0.034632,0.334026,0.022727,0.063706,5,5,6.77433,79.9416,3.301761,462,16,0.00000,198,154.32
4,10.5,0.153072,0.017937,0.008235,0.026831,0.153002,0.481572,0.165972,0.012899,0.066468,7,7,6.97968,79.9130,2.588542,814,392,12.36920,69,147.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8571,10.5,0.204960,0.016396,0.004872,0.057485,0.210688,0.248984,0.211054,0.006094,0.026317,21,21,6.85103,79.9567,3.934272,1723,429,24.83332,3,388.48
8572,10.5,0.268033,0.006633,0.006633,0.000000,0.252728,0.058055,0.275653,0.007620,0.026899,21,22,6.91293,79.9656,7.517433,1378,80,0.00000,125,379.85
8573,10.5,0.236856,0.024311,0.018519,0.022043,0.214297,0.133971,0.261975,0.025120,0.085165,22,22,6.85718,79.9081,2.057225,418,56,3.28440,93,112.79
8574,10.5,0.128607,0.019444,0.004841,0.056061,0.133736,0.341646,0.135153,0.006546,0.040547,22,22,6.91289,79.8846,3.900888,1604,548,31.67440,17,248.46


In [8]:
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import xgboost

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
from keras import backend as K

from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [9]:
train_X, val_X, train_y, val_y = train_test_split(train_X_, train_y_, test_size = 0.2, train_size = 0.8, random_state = 0)

In [11]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    mse = mean_squared_error(test_labels, predictions)
    mae = mean_absolute_error(test_labels, predictions)
    rmse = np.sqrt(mse)
    score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    print('Model Performance')
    print('Accuracy:%f'%accuracy)
    print("Mean cross validation score:%f"%score.mean())
    print('Mean Squared Error : %.4f' % mse)
    print('Root MSE : %.4f' % rmse)

In [ ]:
#random forest

In [12]:
random_forest_model = RandomForestRegressor(n_estimators=100, random_state = 0)
random_forest_model.fit(train_X, train_y)

evaluate(random_forest_model, val_X, val_y)

pred_classes = random_forest_model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)


Model Performance
Accuracy:0.956345
Mean cross validation score:0.499015
Mean Squared Error : 0.0366
Root MSE : 0.1914
F1 score: 0.959347


In [13]:
my_decision_tree_model = DecisionTreeRegressor(min_samples_split=100,
        max_features="auto", random_state=50, 
        max_depth=100)
my_decision_tree_model.fit(train_X, train_y)

evaluate(my_decision_tree_model, val_X, val_y)

pred_classes = my_decision_tree_model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Model Performance
Accuracy:0.941793
Mean cross validation score:0.323674
Mean Squared Error : 0.0471
Root MSE : 0.2170
F1 score: 0.960211


In [ ]:
#ada boost

In [14]:
Ada_boost_model = AdaBoostRegressor()
Ada_boost_model.fit(train_X, train_y)

evaluate(Ada_boost_model, val_X, val_y)

pred_classes = Ada_boost_model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Model Performance
Accuracy:0.940338
Mean cross validation score:0.061829
Mean Squared Error : 0.0914
Root MSE : 0.3023
F1 score: 0.951000


In [15]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1, penalty='l1', solver='liblinear')
param_grid = {'C':[0.001,.009,0.01,.09,1,5,10,25]}
'''{
    'n_estimators' : [10, 200],
    'max_features' : ['auto', 'sqrt', 'log2', 0.5]
}'''
gs_model = GridSearchCV(clf, param_grid = param_grid, scoring = 'recall')
gs_model.fit(train_X, train_y)

predictions = gs_model.predict(val_X)
accuracy = accuracy_score(val_y, predictions.round())
print('Accuracy:%f'%accuracy)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Accuracy:0.911816


In [16]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [18]:
from keras.layers import BatchNormalization
model = Sequential()
model.add(Dense(15))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1,activation='softmax'))

model.compile(optimizer='adam',loss='mse', metrics=['accuracy',f1_m])
#model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

#estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=10, verbose=0)

kfold = StratifiedKFold(n_splits=10, shuffle=True)

#results = cross_val_score(estimator, train_X, train_y, cv=kfold)
#print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
model.fit(train_X_.values, train_y_.values,
          batch_size=20,
          epochs=5,
          validation_split = 0.2,
          verbose = 1,
          shuffle=True)

#evaluate(model, val_X, val_y)
pred_classes = model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Epoch 1/5
687/687 [==============================] - 4s 6ms/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9460 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
Epoch 2/5
687/687 [==============================] - 4s 6ms/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9461 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
Epoch 3/5
687/687 [==============================] - 4s 6ms/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9460 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
Epoch 4/5
687/687 [==============================] - 4s 5ms/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9461 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
Epoch 5/5
687/687 [==============================] - 4s 5ms/step - loss: 0.0999 - accuracy: 0.9001 - f1_m: 0.9461 - val_loss: 0.0896 - val_accuracy: 0.9104 - val_f1_m: 0.9517
F1 score: 0.951000


In [21]:

xg_model = xgboost.XGBClassifier()

score=cross_val_score(xg_model, train_X, train_y, cv=10)
xg_model.fit(train_X, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
def xg_evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    #mse = mean_squared_error(test_labels, predictions)
    #mae = mean_absolute_error(test_labels, predictions)
    #rmse = np.sqrt(mse)
    #score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    #print('Model Performance')
    print('Accuracy:%f'%accuracy)
    #print("Mean cross validation score:%f"%score.mean())
    #print('Mean Squared Error : %.4f' % mse)
    #print('Root MSE : %.4f' % rmse)
xg_evaluate(xg_model, val_X, val_y)
pred_classes = xg_model.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Accuracy:0.954016
F1 score: 0.974881


In [23]:
from sklearn.neighbors import KNeighborsClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=7)
knn_classifier.fit(train_X, train_y)

def knn_evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    #mse = mean_squared_error(test_labels, predictions)
    #mae = mean_absolute_error(test_labels, predictions)
    #rmse = np.sqrt(mse)
    #score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    #print('Model Performance')
    print('Accuracy:%f'%accuracy)
    #print("Mean cross validation score:%f"%score.mean())
    #print('Mean Squared Error : %.4f' % mse)
    #print('Root MSE : %.4f' % rmse)
knn_evaluate(knn_classifier, val_X, val_y)

pred_classes = knn_classifier.predict(val_X)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)
f1 = f1_score(val_y, pred_classes)
print('F1 score: %f' % f1)

Accuracy:0.946740
F1 score: 0.971140
